## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSSS'
meas='SNOW'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
70
+-----------------+------------------+-------------------+---------+--------+--------+---------+-----------+--------------------+--------------------+--------------------+------------------+-----------+---------+------+--------------------+------+
|          coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|               res_1|               res_2|               res_3|          res_mean|    station|total_var|undefs|              vector|  year|
+-----------------+------------------+-------------------+---------+--------+--------+---------+-----------+--------------------+--------------------+--------------------+------------------+-----------+---------+------+--------------------+------+
|50.32407285438648|1.7857347610118768|0.29591995717001485|     73.2|BSBSSSSS| 30.9139| -83.8611|       SNOW|0.001407708945797733|1.503173607490325...|1.157882689766143...|0.9750412820

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_3'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_3 FROM weather


station    year  coeff_3
0  USC00098666  1958.0  0.29592
1  USC00092153  1958.0  0.29592
2  USC00090406  1958.0  0.29592
3  USC00080804  1958.0  0.29592
4  USC00013761  2010.0  0.29592

In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

station  USC00010008  USC00011865  USC00012377  USC00013761  USC00080804  \
year                                                                       
1899.0           NaN          NaN          NaN          NaN          NaN   
1912.0           NaN          NaN          NaN          NaN          NaN   
1934.0           NaN          NaN          NaN          NaN          NaN   
1935.0           NaN          NaN          NaN          NaN          NaN   
1943.0           NaN          NaN          NaN          NaN          NaN   
1954.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN          NaN   
1958.0      0.025788    -0.250708          NaN    -0.234393      0.29592   
1968.0     -0.242983          NaN          NaN          NaN          NaN   
1973.0           NaN          NaN          NaN     0.877269          NaN   

station  USC00081544  USC00083230  USC00085372  USC00085879  USC00087025  \
year                                                                       
1899.0           NaN          NaN          NaN          NaN          NaN   
1912.0           NaN          NaN          NaN          NaN          NaN   
1934.0           NaN          NaN          NaN          NaN          NaN   
1935.0           NaN          NaN          NaN          NaN          NaN   
1943.0           NaN          NaN          NaN          NaN          NaN   
1954.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN    -0.239293          NaN          NaN   
1958.0      0.160854          NaN          NaN     0.555662     0.555662   
1968.0           NaN          NaN          NaN          NaN          NaN   
1973.0           NaN     0.038405          NaN    -0.092327          NaN   

station     ...       USC00095585  USC00096087  USC00097276  USC00098666  \
year        ...                                                            
1899.0      ...          0.056957          NaN          NaN          NaN   
1912.0      ...               NaN          NaN          NaN          NaN   
1934.0      ...               NaN          NaN          NaN          NaN   
1935.0      ...               NaN          NaN          NaN          NaN   
1943.0      ...               NaN          NaN          NaN          NaN   
1954.0      ...               NaN          NaN          NaN          NaN   
1955.0      ...               NaN          NaN          NaN          NaN   
1958.0      ...               NaN     0.160854     0.430986      0.29592   
1968.0      ...               NaN          NaN          NaN          NaN   
1973.0      ...               NaN          NaN          NaN          NaN   

station  USC00098703  USW00003818  USW00012832  USW00013815  USW00093805  \
year                                                                       
1899.0           NaN          NaN          NaN          NaN          NaN   
1912.0           NaN          NaN          NaN          NaN          NaN   
1934.0     -0.235848          NaN          NaN          NaN          NaN   
1935.0           NaN          NaN          NaN          NaN          NaN   
1943.0           NaN          NaN          NaN          NaN          NaN   
1954.0           NaN          NaN          NaN          NaN          NaN   
1955.0           NaN          NaN          NaN          NaN          NaN   
1958.0      0.056957    -0.287621    -0.265522    -0.098888    -0.194247   
1968.0           NaN          NaN          NaN          NaN          NaN   
1973.0           NaN          NaN          NaN     0.238369    -0.125010   

station  USW00093861  
year                  
1899.0           NaN  
1912.0           NaN  
1934.0           NaN  
1935.0           NaN  
1943.0           NaN  
1954.0           NaN  
1955.0           NaN  
1958.0       0.29592  
1968.0           NaN  
1973.0           NaN  

[10 rows x 34 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  23.4865718258
RMS removing mean-by-station=  17.5041171138
RMS removing mean-by-year   =  14.2278526105


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 23.4865718258
0 after removing mean by year    = 14.2278526105
0 after removing mean by stations= 10.4498952444
1 after removing mean by year    = 10.3763765207
1 after removing mean by stations= 10.3513097294
2 after removing mean by year    = 10.3408069946
2 after removing mean by stations= 10.3358444612
3 after removing mean by year    = 10.3333052465
3 after removing mean by stations= 10.3319220139
4 after removing mean by year    = 10.3311270681
4 after removing mean by stations= 10.3306486057
